In [ ]:
#import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from bs4 import BeautifulSoup
from scipy.interpolate import make_interp_spline
import requests
import datetime
import re


In [ ]:
# create the base url which i can use as a base and add the dates to so i can access the different pages
base_url = "https://www.transfermarkt.co.uk/premier-league/marktwerteverein/wettbewerb/GB1/stichtag/"

# Dictionary to store DataFrames for each date
dfs_prem_value = {}

# Custom headers to mimic a real browser
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:109.0) Gecko/20100101 Firefox/109.0",
    "Accept-Language": "en-US,en;q=0.5"
}

# Loop through each year from 2011 to 2024
for year in range(2011, 2025):
    date_str = f"{year}-03-15"
    url = base_url + date_str 
    print(f"Scraping: {url}")
    
    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        print(f"Error fetching page for {date_str}: {response.status_code}")
        continue

    soup = BeautifulSoup(response.text, 'html.parser')
    
    # First, try to find the table with class "items"
    table = soup.find('table', class_="items")
    
    # If not found, look inside HTML comments
    if not table:
        comments = soup.find_all(string=lambda text: isinstance(text, Comment))
        for comment in comments:
            comment_soup = BeautifulSoup(comment, "html.parser")
            table = comment_soup.find('table', class_="items")
            if table:
                break

    if table:
        try:
            df = pd.read_html(str(table))[0]
            dfs_prem_value[date_str] = df
            print(f"Found items table for {date_str} with {len(df)} rows.")
        except Exception as e:
            print(f"Error parsing table for {date_str}: {e}")
    else:
        print(f"No items table found for {date_str}.")

In [ ]:
# Define a regex pattern_top6 matching the desired club names in lower-case
pattern_top6 = r'manchester city|manchester united|chelsea fc|tottenham hotspur|liverpool fc|arsenal fc'
pattern_bottom6 = r'southampton fc|leicester city|ipswich town|fulham fc|everton fc|wolverhampton wanderers'

# Dictionary to store the filtered DataFrames
filtered_dataframes_top_6 = {}

# Loop through each date and DataFrame in the scraped data
for date, df in dfs_prem_value.items():
    # Check that the expected column is present
    if 'Club' in df.columns:
        # Filter rows by matching the pattern_top6 in a case-insensitive manner.
        filtered_df_top = df[df['Club'].str.lower().str.contains(pattern_top6, na=False)]
        filtered_dataframes_top_6[date] = filtered_df_top
        print(f"{date}: {len(filtered_df_top)} rows retained.")
    else:
        print(f"DataFrame for {date} does not contain a 'Club' column. Available columns: {df.columns}")


# Dictionary to store the filtered DataFrames
filtered_dataframes_bottom_6 = {}

# Loop through each date and DataFrame in your scraped data
for date, df in dfs_prem_value.items():
    # Check that the expected column is present
    if 'Club' in df.columns:
        # Filter rows by matching the pattern_top6 in a case-insensitive manner.
        filtered_df_bottom = df[df['Club'].str.lower().str.contains(pattern_bottom6, na=False)]
        filtered_dataframes_bottom_6[date] = filtered_df_bottom
        print(f"{date}: {len(filtered_df_bottom)} rows retained.")
    else:
        print(f"DataFrame for {date} does not contain a 'Club' column. Available columns: {df.columns}")


In [ ]:
filtered_dataframes_top_6['2022-03-15']

In [ ]:
filtered_dataframes_bottom_6['2022-03-15']

In [ ]:
# List of column names to drop
columns_to_drop = ['#', 'wappen', 'Club.1', 'Current value', '%', 'Unnamed: 8','Unnamed: 9' ]  # Replace with your actual column names

# Loop through each DataFrame in your dictionary (e.g., filtered_dataframes)
for date, df in filtered_dataframes_top_6.items():
    # Drop the columns and update the DataFrame in the dictionary
    # Using errors='ignore' ensures that if a column is missing, it won't raise an error
    filtered_dataframes_top_6[date] = df.drop(columns=columns_to_drop, errors='ignore')

for date, df in filtered_dataframes_bottom_6.items():
    # Drop the columns and update the DataFrame in the dictionary
    # Using errors='ignore' ensures that if a column is missing, it won't raise an error
    filtered_dataframes_bottom_6[date] = df.drop(columns=columns_to_drop, errors='ignore')

In [ ]:
# Assuming your dictionary of DataFrames is called 'filtered_dataframes'
for date, df in filtered_dataframes_top_6.items():
    # Check if the column 'League' exists and then rename it to the date
    if 'League' in df.columns:
        df.rename(columns={'League': "Value_" + date}, inplace=True)
    else:
        print(f"'League' column not found in DataFrame for {date}")

for date, df in filtered_dataframes_bottom_6.items():
    # Check if the column 'League' exists and then rename it to the date
    if 'League' in df.columns:
        df.rename(columns={'League': "Value_" + date}, inplace=True)
    else:
        print(f"'League' column not found in DataFrame for {date}")



In [ ]:
filtered_dataframes_top_6['2014-03-15'].head(6)

In [ ]:
filtered_dataframes_bottom_6['2014-03-15'].head(6)

In [ ]:
# New dictionary to store the subset DataFrames
filtered_dataframes_top_6_v1 = {}

for date, df in filtered_dataframes_top_6.items():
    # Construct the value column name based on the date
    value_col = "Value_" + date
    if 'Club' in df.columns and value_col in df.columns:
        # Select only the 'Club' and the 'Value_(date)' columns
        subset_df = df[['Club', value_col]].copy()
        filtered_dataframes_top_6_v1[date] = subset_df
        print(f"For {date}: Retained columns: {subset_df.columns.tolist()}")
    else:
        print(f"DataFrame for {date} does not have the required columns: 'Club' and {value_col}")

filtered_dataframes_bottom_6_v1 = {}

for date, df in filtered_dataframes_bottom_6.items():
    # Construct the value column name based on the date
    value_col = "Value_" + date
    if 'Club' in df.columns and value_col in df.columns:
        # Select only the 'Club' and the 'Value_(date)' columns
        subset_df = df[['Club', value_col]].copy()
        filtered_dataframes_bottom_6_v1[date] = subset_df
        print(f"For {date}: Retained columns: {subset_df.columns.tolist()}")
    else:
        print(f"DataFrame for {date} does not have the required columns: 'Club' and {value_col}")

In [ ]:
filtered_dataframes_top_6_v1['2017-03-15'].head(10)

In [ ]:
filtered_dataframes_bottom_6_v1['2017-03-15'].head(10)

In [ ]:
# Start with an empty combined dataframe
combined_df_top6 = None
combined_df_bottom6 = None

# Loop through each date and merge on the 'Club' column
for date, df in filtered_dataframes_top_6_v1.items():
    if combined_df_top6 is None:
        combined_df_top6 = df
    else:
        combined_df_top6 = pd.merge(combined_df_top6, df, on='Club', how='outer')

# Display the first few rows of the combined dataframe
print(combined_df_top6.head())

for date, df in filtered_dataframes_bottom_6_v1.items():
    if combined_df_bottom6 is None:
        combined_df_bottom6 = df
    else:    
        combined_df_bottom6 = pd.merge(combined_df_bottom6, df, on='Club', how='outer') 
print(combined_df_bottom6.head())


In [ ]:

def convert_value(value_str):
    """
    Convert a monetary string (e.g., "€310.75m", "€1.19bn") into a numeric value.
    """
    if isinstance(value_str, str):
        # Remove the euro symbol and extra spaces, then convert to lower case
        value_str = value_str.replace("€", "").strip().lower()
        if "m" in value_str:
            try:
                # Remove "m", convert to float, and multiply by 1e6
                return float(value_str.replace("m", "")) * 1_000_000
            except:
                return None
        elif "bn" in value_str:
            try:
                # Remove "bn", convert to float, and multiply by 1e9
                return float(value_str.replace("bn", "")) * 1_000_000_000
            except:
                return None
        else:
            try:
                return float(value_str)
            except:
                return None
    return value_str

# Assuming your combined dataframe is named 'combined_df_top6'
# Loop through all columns and apply conversion on columns that start with "Value_"
for col in combined_df_top6.columns:
    if col.startswith("Value_"):
        combined_df_top6[col] = combined_df_top6[col].apply(convert_value)

for col in combined_df_bottom6.columns:
    if col.startswith("Value_"):
        combined_df_bottom6[col] = combined_df_bottom6[col].apply(convert_value)  




In [ ]:
df_transposed_top6 = combined_df_top6.set_index('Club').transpose()

# Remove the "Value_" prefix from the index and convert to datetime objects.
df_transposed_top6.index = pd.to_datetime(df_transposed_top6.index.str.replace("Value_", "", regex=True))
df_transposed_top6 = df_transposed_top6.sort_index()

df_transposed_bottom6 = combined_df_bottom6.set_index('Club').transpose()

# Remove the "Value_" prefix from the index and convert to datetime objects.
df_transposed_bottom6.index = pd.to_datetime(df_transposed_bottom6.index.str.replace("Value_", "", regex=True))
df_transposed_bottom6 = df_transposed_bottom6.sort_index()

In [ ]:
# ---------------------- Custom Y-Axis Formatter ---------------------- #
def custom_y_formatter(x, pos):
    if x < 1e9:
        return f"{x/1e6:,.0f} million"
    else:
        return f"{x/1e9:,.1f} billion"

# Set Seaborn style and palette for aesthetics.
sns.set_style("whitegrid")
sns.set_palette("Set2")

# Create a figure with 2 subplots (vertical layout).
fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(12, 10), sharex=True)

# ---------------------- Plotting Function ---------------------- #
def plot_data(ax, df_transposed, title):
    # Convert datetime index to numeric for spline interpolation.
    x_dates = mdates.date2num(df_transposed.index.to_pydatetime())
    
    # Plot each club's data.
    for club in df_transposed.columns:
        y = df_transposed[club].values
        if len(x_dates) >= 3:
            spline = make_interp_spline(x_dates, y, k=3)  # Cubic spline for smoothness.
            x_dense = np.linspace(x_dates.min(), x_dates.max(), 300)
            y_smooth = spline(x_dense)
            x_dense_dates = mdates.num2date(x_dense)
            ax.plot(x_dense_dates, y_smooth, label=club, linewidth=2)
        else:
            ax.plot(df_transposed.index, y, marker='o', label=club, linewidth=2)
    
    # Format the x-axis: one tick per year, display only the year.
    ax.xaxis.set_major_locator(mdates.YearLocator())
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y'))
    ax.tick_params(axis='x', rotation=45)
    
    # Disable scientific notation/offsets on the y-axis.
    ax.ticklabel_format(axis='y', style='plain', useOffset=False)
    ax.get_yaxis().get_major_formatter().set_scientific(False)
    
    # Limit the number of y-axis ticks.
    ax.yaxis.set_major_locator(ticker.MaxNLocator(6))
    
    # Apply the custom y-axis formatter.
    ax.yaxis.set_major_formatter(ticker.FuncFormatter(custom_y_formatter))
    
    # Set titles and labels.
    ax.set_title(title, fontsize=14, fontweight='bold')
    ax.set_xlabel("Year", fontsize=12)
    ax.set_ylabel("Investment Value", fontsize=12)
    ax.legend(loc='upper left', bbox_to_anchor=(1.02, 1), borderaxespad=0)

# ---------------------- Plot for Top 6 Teams ---------------------- #
plot_data(axes[0], df_transposed_top6, "Top 6 Teams Values Over Time (2011-2024)")

# ---------------------- Plot for Bottom 6 Teams ---------------------- #
plot_data(axes[1], df_transposed_bottom6, "Bottom 6 Teams Values Over Time (2011-2024)")

plt.tight_layout()
plt.show()


how this compares to the top 5 leagues in eurpope.

In [ ]:
# Dictionary mapping leagues to their base URL
base_urls = {
    "La Liga": "https://www.transfermarkt.co.uk/la-liga/marktwerteverein/wettbewerb/ES1/stichtag/",
    "Bundesliga": "https://www.transfermarkt.co.uk/bundesliga/marktwerteverein/wettbewerb/L1/stichtag/",
    "Ligue 1": "https://www.transfermarkt.co.uk/ligue-1/marktwerteverein/wettbewerb/FR1/stichtag/",
    "Serie A": "https://www.transfermarkt.co.uk/serie-a/marktwerteverein/wettbewerb/IT1/stichtag/",
    "Premier League": "https://www.transfermarkt.co.uk/premier-league/marktwerteverein/wettbewerb/GB1/stichtag/"
}

# Dictionary to store DataFrames for each league and date
dfs_league = {}

# Custom headers to mimic a real browser
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:109.0) Gecko/20100101 Firefox/109.0",
    "Accept-Language": "en-US,en;q=0.5"
}

# Loop through each league
for league, base_url in base_urls.items():
    dfs_league[league] = {}  # create a sub-dictionary for this league
    print(f"\nScraping data for {league}:")
    
    # Loop through each year from 2011 to 2024
    for year in range(2011, 2025):
        date_str = f"{year}-03-15"
        url = base_url + date_str
        print(f"Scraping: {url}")
    
        response = requests.get(url, headers=headers)
        if response.status_code != 200:
            print(f"Error fetching page for {league} {date_str}: {response.status_code}")
            continue

        soup = BeautifulSoup(response.text, 'html.parser')
    
        # Try to find the table with class "items"
        table = soup.find('table', class_="items")
    
        # If not found, look inside HTML comments
        if not table:
            comments = soup.find_all(string=lambda text: isinstance(text, Comment))
            for comment in comments:
                comment_soup = BeautifulSoup(comment, "html.parser")
                table = comment_soup.find('table', class_="items")
                if table:
                    break

        if table:
            try:
                df = pd.read_html(str(table))[0]
                dfs_league[league][date_str] = df
                print(f"Found items table for {league} {date_str} with {len(df)} rows.")
            except Exception as e:
                print(f"Error parsing table for {league} {date_str}: {e}")
        else:
            print(f"No items table found for {league} {date_str}.")


In [ ]:
# List of column names to drop
columns_to_drop = ['#', 'wappen', 'Club.1', 'Current value', '%', 'Unnamed: 8','Unnamed: 9' ]  # Replace with your actual column names

# Loop through each league and its corresponding DataFrames in the nested dictionary
for league, league_data in dfs_league.items():
    for date, df in league_data.items():
        # Drop the columns and update the DataFrame in the nested dictionary
        # Using errors='ignore' ensures that if a column is missing, it won't raise an error
        dfs_league[league][date] = df.drop(columns=columns_to_drop, errors='ignore')



# Loop through each league and its corresponding DataFrames in the nested dictionary
for league, league_data in dfs_league.items():
    for date, df in league_data.items():
        # Check if the column 'League' exists and then rename it to the date
        if 'League' in df.columns:
            df.rename(columns={'League': "Value_" + date}, inplace=True)
        else:
            print(f"'League' column not found in DataFrame for {league} on {date}")

# New dictionary to store the subset DataFrames
dfs_league_cleaned = {}

for league, league_data in dfs_league.items():
    for date, df in league_data.items():
        # Construct the value column name based on the date
        value_col = "Value_" + date
        if 'Club' in df.columns and value_col in df.columns:
            # Select only the 'Club' and the 'Value_(date)' columns
            subset_df = df[['Club', value_col]].copy()
            if league not in dfs_league_cleaned:
                dfs_league_cleaned[league] = {}
            dfs_league_cleaned[league][date] = subset_df

# Start with an empty combined dataframe

combined_la_liga = None
combined_ligue_1 = None
combined_bundesliga = None
combined_serie_a = None
combined_premier_league = None

# Loop through each league and its corresponding DataFrames in the nested dictionary  
for league, league_data in dfs_league_cleaned.items():
    for date, df in league_data.items():
        if league == 'La Liga':
            if combined_la_liga is None:
                combined_la_liga = df
            else:
                combined_la_liga = pd.merge(combined_la_liga, df, on='Club', how='outer')
        elif league == 'Ligue 1':
            if combined_ligue_1 is None:
                combined_ligue_1 = df
            else:
                combined_ligue_1 = pd.merge(combined_ligue_1, df, on='Club', how='outer')
        elif league == 'Bundesliga':
            if combined_bundesliga is None:
                combined_bundesliga = df
            else:
                combined_bundesliga = pd.merge(combined_bundesliga, df, on='Club', how='outer')
        elif league == 'Serie A':
            if combined_serie_a is None:
                combined_serie_a = df
            else:
                combined_serie_a = pd.merge(combined_serie_a, df, on='Club', how='outer')
        elif league == 'Premier League':
            if combined_premier_league is None:
                combined_premier_league = df
            else:
                combined_premier_league = pd.merge(combined_premier_league, df, on='Club', how='outer')




# realised that there was a unknown row in the combined dataframe that was causing issues with the plotting
combined_ligue_1_clean = combined_ligue_1.drop(18)
combined_la_liga_clean = combined_la_liga.drop(17)
combined_bundesliga_clean = combined_bundesliga.drop(15)
combined_serie_a_clean = combined_serie_a.drop(17)
combined_premier_league_clean = combined_premier_league.drop(17)


def convert_value(value_str):
    """
    Convert a monetary string (e.g., "€310.75m", "€1.19bn") into a numeric value.
    """
    if isinstance(value_str, str):
        # Remove the euro symbol and extra spaces, then convert to lower case
        value_str = value_str.replace("€", "").strip().lower()
        if "m" in value_str:
            try:
                # Remove "m", convert to float, and multiply by 1e6
                return float(value_str.replace("m", "")) * 1_000_000
            except:
                return None
        elif "bn" in value_str:
            try:
                # Remove "bn", convert to float, and multiply by 1e9
                return float(value_str.replace("bn", "")) * 1_000_000_000
            except:
                return None
        else:
            try:
                return float(value_str)
            except:
                return None
    return value_str

# List of league DataFrames
league_dfs1 = [
    combined_ligue_1_clean,
    combined_la_liga_clean,
    combined_bundesliga_clean,
    combined_serie_a_clean,
    combined_premier_league_clean
]

# Loop through each league DataFrame and convert the columns starting with "Value_"
for df in league_dfs1:
    for col in df.columns:
        if col.startswith("Value_"):
            df[col] = df[col].apply(convert_value)


league_dfs1[0] = combined_ligue_1_clean
league_dfs1[1] = combined_la_liga_clean
league_dfs1[2] = combined_bundesliga_clean
league_dfs1[3] = combined_serie_a_clean
league_dfs1[4] = combined_premier_league_clean

# Transpose and prepare data for Ligue 1
df_transposed_ligue_1 = combined_ligue_1_clean.set_index('Club').transpose()
df_transposed_ligue_1.index = pd.to_datetime(df_transposed_ligue_1.index.str.replace("Value_", "", regex=True))
df_transposed_ligue_1 = df_transposed_ligue_1.sort_index()

# Transpose and prepare data for La Liga
df_transposed_la_liga = combined_la_liga_clean.set_index('Club').transpose()
df_transposed_la_liga.index = pd.to_datetime(df_transposed_la_liga.index.str.replace("Value_", "", regex=True))
df_transposed_la_liga = df_transposed_la_liga.sort_index()

# Transpose and prepare data for Bundesliga
df_transposed_bundesliga = combined_bundesliga_clean.set_index('Club').transpose()
df_transposed_bundesliga.index = pd.to_datetime(df_transposed_bundesliga.index.str.replace("Value_", "", regex=True))
df_transposed_bundesliga = df_transposed_bundesliga.sort_index()

# Transpose and prepare data for Serie A
df_transposed_serie_a = combined_serie_a_clean.set_index('Club').transpose()
df_transposed_serie_a.index = pd.to_datetime(df_transposed_serie_a.index.str.replace("Value_", "", regex=True))
df_transposed_serie_a = df_transposed_serie_a.sort_index()

# Transpose and prepare data for Premier League
df_transposed_premier_league = combined_premier_league_clean.set_index('Club').transpose()
df_transposed_premier_league.index = pd.to_datetime(df_transposed_premier_league.index.str.replace("Value_", "", regex=True))
df_transposed_premier_league = df_transposed_premier_league.sort_index()


In [ ]:
# ---------------------- Custom Y-Axis Formatter ---------------------- #
def custom_y_formatter(x, pos):
    if x < 1e9:
        return f"{x/1e6:,.0f} million"
    else:
        return f"{x/1e9:,.1f} billion"

# ---------------------- Set Seaborn Style ---------------------- #
sns.set_style("whitegrid")
sns.set_palette("Set2")

# ---------------------- Function to Plot League Average ---------------------- #
def plot_league_avg(ax, df, label):
    # Compute the average value across all clubs for each date (index)
    avg_series = df.mean(axis=1)
    # Convert the datetime index to numeric values for spline interpolation.
    x_dates = mdates.date2num(avg_series.index.to_pydatetime())
    y = avg_series.values
    
    # If there are at least 3 points, use cubic spline interpolation for a smooth curve.
    if len(x_dates) >= 3:
        spline = make_interp_spline(x_dates, y, k=3)
        x_dense = np.linspace(x_dates.min(), x_dates.max(), 300)
        y_smooth = spline(x_dense)
        x_dense_dates = mdates.num2date(x_dense)
        ax.plot(x_dense_dates, y_smooth, label=label, linewidth=2)
    else:
        ax.plot(avg_series.index, y, marker='o', label=label, linewidth=2)

# ---------------------- Create Figure and Axis ---------------------- #
fig, ax = plt.subplots(figsize=(12, 6))

# ---------------------- Plot the Average for Each League ---------------------- #
plot_league_avg(ax, df_transposed_ligue_1, "Ligue 1")
plot_league_avg(ax, df_transposed_la_liga, "La Liga")
plot_league_avg(ax, df_transposed_bundesliga, "Bundesliga")
plot_league_avg(ax, df_transposed_serie_a, "Serie A")
plot_league_avg(ax, df_transposed_premier_league, "Premier League")

# ---------------------- Highlight Significant Financial Events ---------------------- #
# Define events with their year and label. Adjust these events as needed.
events = [
    {"year": 2011, "label": "UEFA FFP introduced"},
    {"year": 2020, "label": "COVID-19 Pandemic"}
]

for event in events:
    # Define the start and end of the event year.
    start_date = datetime.datetime(event['year'], 1, 1)
    end_date = datetime.datetime(event['year'], 12, 31)
    # Shade the entire year with a semi-transparent gray.
    ax.axvspan(start_date, end_date, color='gray', alpha=0.2)
    # Add a vertical annotation for the event.
    ax.text(start_date, ax.get_ylim()[1]*0.95, event['label'], rotation=90,
            verticalalignment='top', fontsize=10, color='black')

# ---------------------- Format X-Axis ---------------------- #
ax.xaxis.set_major_locator(mdates.YearLocator())
ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y'))
ax.tick_params(axis='x', rotation=45)

# ---------------------- Format Y-Axis ---------------------- #
ax.ticklabel_format(axis='y', style='plain', useOffset=False)
ax.get_yaxis().get_major_formatter().set_scientific(False)
ax.yaxis.set_major_locator(ticker.MaxNLocator(6))
ax.yaxis.set_major_formatter(ticker.FuncFormatter(custom_y_formatter))

# ---------------------- Set Title, Labels, and Legend ---------------------- #
ax.set_title("Average of Club's values in Europes top 5 leagues (2011-2024)", fontsize=14, fontweight='bold')
ax.set_xlabel("Year", fontsize=12)
ax.set_ylabel("Investment Value", fontsize=12)
ax.legend(loc='upper left', bbox_to_anchor=(1.02, 1), borderaxespad=0)

plt.tight_layout()
plt.show()


In [ ]:
import requests
import pandas as pd
from bs4 import BeautifulSoup, Comment

# Dictionary mapping leagues to their revenue/expenditure URL
base_urls = {
    "Premier League": "https://www.transfermarkt.co.uk/premier-league/einnahmenausgaben/wettbewerb/GB1/plus/0?ids=a&sa=&saison_id=&saison_id_bis=2024&nat=&pos=&altersklasse=&w_s=&leihe=&intern=0",
    "La Liga": "https://www.transfermarkt.co.uk/laliga/einnahmenausgaben/wettbewerb/ES1/plus/0?ids=a&sa=&saison_id=&saison_id_bis=2024&nat=&pos=&altersklasse=&w_s=&leihe=&intern=0",
    "Serie A": "https://www.transfermarkt.co.uk/serie-a/einnahmenausgaben/wettbewerb/IT1/plus/0?ids=a&sa=&saison_id=&saison_id_bis=2024&nat=&pos=&altersklasse=&w_s=&leihe=&intern=0",
    "Bundesliga": "https://www.transfermarkt.co.uk/bundesliga/einnahmenausgaben/wettbewerb/L1/plus/0?ids=a&sa=&saison_id=&saison_id_bis=2024&nat=&pos=&altersklasse=&w_s=&leihe=&intern=0",
    "Ligue 1": "https://www.transfermarkt.co.uk/ligue-1/einnahmenausgaben/wettbewerb/FR1/plus/0?ids=a&sa=&saison_id=&saison_id_bis=2024&nat=&pos=&altersklasse=&w_s=&leihe=&intern=0"
}

# Dictionary to store DataFrames for each league
dfs_income_vs_expenditure = {}

# Custom headers to mimic a real browser
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:109.0) Gecko/20100101 Firefox/109.0",
    "Accept-Language": "en-US,en;q=0.5"
}

# Loop through each league URL and scrape the table
for league, url in base_urls.items():
    print(f"\nScraping data for {league}:")
    print(f"Scraping: {url}")
    
    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        print(f"Error fetching page for {league}: {response.status_code}")
        continue

    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Try to find the table with class "items"
    table = soup.find('table', class_="items")
    
    # If not found, look inside HTML comments
    if not table:
        comments = soup.find_all(string=lambda text: isinstance(text, Comment))
        for comment in comments:
            comment_soup = BeautifulSoup(comment, "html.parser")
            table = comment_soup.find('table', class_="items")
            if table:
                break

    if table:
        try:
            df = pd.read_html(str(table))[0]
            dfs_income_vs_expenditure[league] = df
            print(f"Found items table for {league} with {len(df)} rows.")
        except Exception as e:
            print(f"Error parsing table for {league}: {e}")
    else:
        print(f"No items table found for {league}.")


In [ ]:
dfs_income_vs_expenditure['Premier League'].head(30)

In [ ]:
# List of column names to drop
columns_to_drop = ['#', 'Club', 'Arrivals', 'Income', 'Departures','Balance' ]

# Loop through each league and its corresponding DataFrames in the nested dictionary
for league, df in dfs_income_vs_expenditure.items():
        # Drop the columns and update the DataFrame in the nested dictionary
        # Using errors='ignore' ensures that if a column is missing, it won't raise an error
        dfs_income_vs_expenditure[league] = df.drop(columns=columns_to_drop, errors='ignore')



In [ ]:
dfs_income_vs_expenditure['Ligue 1'].head(30)

In [ ]:
# Loop through each league and its corresponding DataFrames in the nested dictionary
for league, df in dfs_income_vs_expenditure.items():
        # Check if the column 'Club.1' exists and then rename it to the date
        if 'Club.1' in df.columns:
            df.rename(columns={'Club.1': "Club"}, inplace=True)
        if 'Club.2' in df.columns:
            df.rename(columns={'Club.2': "Club's Expenditure"}, inplace=True)
        if 'Expenditure' in df.columns:
            df.rename(columns={'Expenditure': "Arrivals"}, inplace=True)



In [ ]:
# only keep the first 15 rows in each league df
# Iterate through each DataFrame in the dictionary and slice it
for league, df in dfs_income_vs_expenditure.items():
	dfs_income_vs_expenditure[league] = df.iloc[:15]


In [ ]:
# Convert the "Club's Expenditure" column to numeric values
for league, df in dfs_income_vs_expenditure.items():
    dfs_income_vs_expenditure[league]["Club's Expenditure"] = dfs_income_vs_expenditure[league]["Club's Expenditure"].apply(convert_value)

In [ ]:
dfs_income_vs_expenditure['La Liga'].head(30)

In [ ]:
# ---------------------- Compute Average Expenditure ---------------------- #
league_avg_expenditure = {}
for league, df in dfs_income_vs_expenditure.items():
    if "Club's Expenditure" in df.columns:
        # Compute the mean expenditure for each league (ignoring missing values)
        league_avg_expenditure[league] = df["Club's Expenditure"].mean()

df_avg = pd.DataFrame(
    list(league_avg_expenditure.items()),
    columns=["League", "AvgExpenditure"]
)
df_avg.sort_values("AvgExpenditure", inplace=True)

# ---------------------- Compute Average Arrivals ---------------------- #
league_avg_arrivals = {}
for league, df in dfs_income_vs_expenditure.items():
    if "Arrivals" in df.columns:
        # Compute the mean arrivals for each league (ignoring missing values)
        league_avg_arrivals[league] = df["Arrivals"].mean()

df_arrivals = pd.DataFrame(
    list(league_avg_arrivals.items()),
    columns=["League", "AvgArrivals"]
)
# Ensure the order of arrivals matches expenditure (using League as key)
df_arrivals = df_arrivals.set_index("League").loc[df_avg["League"]].reset_index()

# ---------------------- Plotting the Bar Chart ---------------------- #
sns.set_theme(style="whitegrid")
plt.figure(figsize=(10, 6))
ax = sns.barplot(x="League", y="AvgExpenditure", data=df_avg, palette="Set2")

ax.set_title("Average Club Expenditure by League with Average Arrivals", fontsize=16)
ax.set_xlabel("League", fontsize=14)
ax.set_ylabel("Average Expenditure (€)", fontsize=14)

# ---------------------- Custom Y-Axis Formatter for Expenditure ---------------------- #
def custom_y_formatter(x, pos):
    if x < 1e9:
        return f"{x/1e6:,.0f}M"
    else:
        return f"{x/1e9:,.1f}B"

ax.yaxis.set_major_formatter(plt.FuncFormatter(custom_y_formatter))
plt.xticks(rotation=45)
plt.tight_layout()

# Annotate each bar with its expenditure value
for p in ax.patches:
    height = p.get_height()
    label = custom_y_formatter(height, 0)
    ax.text(
        p.get_x() + p.get_width() / 2,
        height - (height * 0.1),
        label,
        ha='center',
        va='top',
        fontsize=12,
        color='black'
    )

# ---------------------- Overlay Smooth Line Plot for Average Arrivals ---------------------- #
# Create a secondary y-axis for arrivals
ax2 = ax.twinx()
ax2.set_ylabel("Average Arrivals (Count)", fontsize=14, color='black')
ax2.tick_params(axis='y', labelcolor='black')
ax2.grid(False)  # Disable gridlines on the secondary axis

# Convert categorical league names to numeric positions
leagues = df_avg["League"].tolist()
x_numeric = np.arange(len(leagues))
y_arrivals = df_arrivals["AvgArrivals"].values

# Create a smooth (soft) line using spline interpolation if possible
if len(x_numeric) >= 3:
    x_dense = np.linspace(x_numeric.min(), x_numeric.max(), 300)
    spline = make_interp_spline(x_numeric, y_arrivals, k=2)  # quadratic spline for smoothing
    y_dense = spline(x_dense)
    ax2.plot(x_dense, y_dense, color='blue', linewidth=1.5, label="Avg Arrivals")
else:
    ax2.plot(x_numeric, y_arrivals, color='blue', marker='o', linewidth=1.5, label="Avg Arrivals")

# Overlay the original data points
ax2.scatter(x_numeric, y_arrivals, color='black', marker='o', s=40)

# Set x-ticks on the secondary axis to match the league names
ax2.set_xticks(x_numeric)
ax2.set_xticklabels(leagues)

# Compute a vertical offset (3% of the y-axis range) to shift the text upward
offset = (ax2.get_ylim()[1] - ax2.get_ylim()[0]) * 0.03

# Annotate each point on the arrivals line with its value (shifted upward)
# Also, shift the annotation horizontally for Premier League
for x_val, y_val, league in zip(x_numeric, y_arrivals, leagues):
    x_offset = 0.0
    if league == "Premier League":
        x_offset = 0.15  # Adjust this value as needed
    ax2.text(x_val + x_offset, y_val + offset, f"{y_val:.1f}", ha='center', va='bottom', fontsize=10, color='black')

# Add a legend for the arrivals line (optional)
ax2.legend(loc='upper left', fontsize=10)

plt.show()
